In [4]:
import requests
import requests.auth
from requests_oauthlib import OAuth1
import oauth2 as oauth
import time
import os
import json
import pprint
import csv

In [5]:
# Import credentials from local source:
reader = csv.reader(open("/Users/mchifala/Desktop/ATLS_5412/Credentials.csv"))
credentials = {}
for line in reader:
    credentials[line[0]] = line[1]
    

In [6]:
# Get a Reddit credentials:
def getRedditHeaders():
    reddit_id = credentials['reddit_id']
    reddit_secret = credentials['reddit_secret']
    reddit_username = credentials['reddit_username']
    reddit_password = credentials['reddit_password']
    reddit_user_agent = credentials['reddit_user_agent']

    reddit_auth = requests.auth.HTTPBasicAuth(reddit_id, reddit_secret)
    reddit_post_data = {'grant_type': 'password', 'username': reddit_username, 'password': reddit_password}
    reddit_token_headers = {'User-Agent': reddit_user_agent}
    r = requests.post("https://www.reddit.com/api/v1/access_token", auth= reddit_auth, data=reddit_post_data, headers=reddit_token_headers)
    #print(r.json())

    reddit_token = r.json()['token_type'] + ' ' + r.json()['access_token']
    
    # Return header dictionary
    return {"Authorization": reddit_token, "User-Agent": reddit_user_agent, 'Content-Type': 'application/json'}


In [7]:
# Get Twitter credentials:
def getTwitterAuth():
    twitter_key = credentials['twitter_key']
    twitter_secret =  credentials['twitter_secret']
    twitter_token = credentials['twitter_token']
    twitter_secret_token = credentials['twitter_secret_token']
    
    #Return authorization object
    return OAuth1(twitter_key, twitter_secret, twitter_token, twitter_secret_token)

getTwitterAuth()

In [8]:
# Get all available Twitter trends:
twitter_auth = getTwitterAuth()
r = requests.get('https://api.twitter.com/1.1/trends/available.json', auth = twitter_auth)
#print(r.json())

In [9]:
# Extract the location ID codes for trends:
world_codes = []
for i in range(0,len(r.json())):
    world_codes.append(r.json()[i]['woeid'])

test_codes = world_codes[0:10]
print(test_codes)

[1, 2972, 3369, 3444, 3534, 4118, 8676, 8775, 9807, 12723]


In [10]:
#Search each location ID for its specific trends:
topics = []
for code in test_codes:
    r2 = requests.get('https://api.twitter.com/1.1/trends/place.json?id='+str(code), auth = twitter_auth)
    for i in range(0,len(r2.json()[0])):
        print(r2.json()[0]['trends'][i]['name'])
        topics.append((r2.json()[0]['trends'][i]['name']))

#UFC235
#nitiasa
#LulaGigante
#Hilbet10TLVeriyor
#UFC235
Diego Sanchez
#VWFC
#1stHabsGoal
#TavaresDayTO
#TFCLive
Boucher
Khloe
#UFC235
#VWFC
#1stHabsGoal
#ThankYouIggy
Montréal
#UFC235
#VWFC
#1stHabsGoal
#TavaresDayTO
#EatEnergize
Don Cherry
#UFC235
#Oilers
#UFC235
#VWFC
#1stHabsGoal
#UFC235
#VWFC
#1stHabsGoal
#ThankYouIggy
#UFC235
#VWFC
#1stHabsGoal
#EatEnergize
#avfc
Brilliant
Leeds
Absolutely


In [12]:
# Make Reddit queries for trends and upload to Elasticsearch:
reddit_headers = getRedditHeaders()
for query in ['#1erButCH']: #set(topics):
    index = 'test_02_23_19'
    start = time.time()
    r_get = requests.get('https://oauth.reddit.com/r/all/search?q='+query+'&t=day&limit=2', headers = reddit_headers)
    r_get.headers['content-type']
    list = r_get.json()['data']['children']
    end = time.time()

    data = []
    fields_to_keep = ['id','title', 'author', 'score', 'subreddit', 'num_comments', 'num_crossposts', 'created_utc']
    for value in list:
        print('hi'+ value)
        temp = {k: value['data'][k] for k in fields_to_keep}
        temp.update({'hashtag': query})
        data.append({'index': {"_index": index, '_type': '_doc'}, '_id': 'r_'+ temp['id'] })
        del temp['id']
        data.append(temp)
        
    data_to_post = '\n'.join(json.dumps(d) for d in data)  
    #print(data_to_post)
    #r_post = requests.post('http://localhost:9200/'+index+'/_bulk',headers = headers, data=data_to_post+'\n')
    
    #print(r_post.text)

In [13]:
# This is a working example: make sure the indexes match
import json
data = [
    { "index" : { "_index" : "test5", "_type" : "type1", "_id" : "1" } },
    { "field1" : "value1" },
    { "delete" : { "_index" : "test5", "_type" : "type1", "_id" : "2" }, },
    { "create" : { "_index" : "test5", "_type" : "type1", "_id" : "3" }, },
    { "field1" : "value3" },
    { "update" : {"_id" : "1", "_type" : "type1", "_index" : "test5"} },
    { "doc" : {"field2" : "value2"} }
]

data_to_post = '\n'.join(json.dumps(d) for d in data)
print(data_to_post)
#r = requests.post('http://localhost:9200/test5/_bulk', headers = headers, data=data_to_post + '\n')
#print(r.text)

{"index": {"_index": "test5", "_type": "type1", "_id": "1"}}
{"field1": "value1"}
{"delete": {"_index": "test5", "_type": "type1", "_id": "2"}}
{"create": {"_index": "test5", "_type": "type1", "_id": "3"}}
{"field1": "value3"}
{"update": {"_id": "1", "_type": "type1", "_index": "test5"}}
{"doc": {"field2": "value2"}}


In [ ]:
# Communicating with Elastic Search:
#r = requests.get('http://localhost:9200/twitter/_doc/TQqw02gBSrHX7MwTUvS-', headers = {'User-agent': 'mchifala'})
requests.post('http://34.73.60.209:9200/hi_yash/_doc/',  json = {"user" : "kimchy",
    "post_date" : "2019-01-28T14:12:12",
    "message" : "Hi Yash"})
